In [66]:
import pandas as pd
import numpy as np
from preprocess import scale
from sklearn.linear_model import LogisticRegression

In [114]:
url = 'https://raw.githubusercontent.com/probml/pmtkdata/master/spamData/spam.data'
data = pd.read_table(url, header = None, sep = ' ')

In [115]:
y = data[57].values
X = data[range(57)].values
del data

In [141]:
train = np.random.choice(len(y), replace = False, size = 3065)
test = np.ones(len(y), dtype=bool)
test[train] = 0

In [145]:
train_y = y[train]
test_y = y[test]
train_X = X[train,]
test_X = X[test,]
del X
del y

In [151]:
clf = LogisticRegression(C = 0.5)

In [160]:
def test(transform, C):
    clf = LogisticRegression(C = C)
    clf.fit(transform(train_X), train_y)
    result = lambda X, y: '%0.3f' % np.mean(1.0 - np.mean(clf.predict(transform(X)) == y))
    print result(train_X, train_y), result(test_X, test_y)

for C in np.arange(0.5, 2, 0.5):
    print C
    test(lambda X: X, C)
    test(lambda X: scale(X), C)
    test(lambda X: np.log(X + 0.1), C)
    test(lambda X:  1 * (X > 0), C)

0.5
0.072 0.079
0.074 0.080
0.054 0.050
0.063 0.074
1.0
0.073 0.079
0.073 0.085
0.054 0.050
0.063 0.072
1.5
0.071 0.078
0.071 0.085
0.054 0.051
0.061 0.073
